In [1]:
!pip install -q tensorflow==2.3

In [2]:
#аугментации изображений
!pip install albumentations -q

In [230]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from datetime import datetime
import random
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import time
from zipfile import ZipFile
from IPython.display import clear_output
from collections import Counter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from pandas_profiling import ProfileReport

from catboost import CatBoostRegressor
from catboost.text_processing import Dictionary, Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
pd.set_option('display.max_columns', None)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

Python       : 3.7.9 | packaged by conda-forge | (default, Dec  9 2020, 21:08:20) 
Numpy        : 1.18.5
Tensorflow   : 2.3.0


In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [7]:
!pip freeze > requirements.txt

# DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [155]:
DATA_DIR = '../input/sfdstcarprice2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [9]:
train.head(10)

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_info,...,price,productionDate,sell_id,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль
0,седан,BMW,чёрный,Авто на бодром ходу. Все работает отлично. П...,3.0 LTR,272 N12,бензин,245000,2007,5ER,...,599000.0,2007,1099980990,SEDAN AUTOMATIC 3.0,автоматическая,3 или более,NaN,Оригинал,полный,Левый
1,седан,AUDI,серебристый,"Продажа от собственника, второй владелец, ПТС ...",2.8 LTR,204 N12,бензин,183000,2011,A6,...,850000.0,2011,1095836906,SEDAN VARIATOR 2.8,вариатор,2 владельца,8 лет и 6 месяцев,Оригинал,передний,Левый
2,седан,MERCEDES,чёрный,Внимание! Только для клиентов AVILON Автомобил...,3.5 LTR,306 N12,бензин,122733,2009,E_KLASSE,...,1325000.0,2013,1100195530,SEDAN AUTOMATIC 3.5,автоматическая,3 или более,NaN,Оригинал,полный,Левый
3,седан,AUDI,белый,В комплекте зимние колёса. 27.07.2020 Замена п...,2.0 LTR,180 N12,бензин,151000,2011,A6,...,815000.0,2011,1099880662,SEDAN VARIATOR 2.0,вариатор,3 или более,4 года и 9 месяцев,Оригинал,передний,Левый
4,лифтбек,AUDI,белый,"Отличный авто, 2011 года выпуска, кроме передн...",1.8 LTR,160 N12,бензин,140000,2007,A5,...,810000.0,2011,1083244610,LIFTBACK VARIATOR 1.8,вариатор,3 или более,2 года и 9 месяцев,Оригинал,передний,Левый
5,седан,MERCEDES,чёрный,Автомобиль в очень хорошем состоянии. Бережная...,1.8 LTR,184 N12,бензин,229000,2006,C_KLASSE,...,550000.0,2007,1049987852,SEDAN AUTOMATIC 1.8,автоматическая,3 или более,6 лет и 9 месяцев,Оригинал,задний,Левый
6,седан,MERCEDES,чёрный,НОВЫЙ АВТОМОБИЛЬ!!!\nАВТОМОБИЛЬ НЕ СТАНОВИЛСЯ ...,2.0 LTR,197 N12,бензин,50,2016,E_KLASSE,...,3170000.0,2020,1099934465,SEDAN AUTOMATIC 2.0,автоматическая,1 владелец,NaN,Оригинал,полный,Левый
7,седан,BMW,чёрный,Идеальное состояние салона и кузова. Не требуе...,3.0 LTR,326 N12,бензин,90000,2015,7ER,...,2900000.0,2016,1098936372,SEDAN AUTOMATIC 3.0,автоматическая,3 или более,4 года и 3 месяца,Оригинал,полный,Левый
8,седан,AUDI,чёрный,Автомобиль технически полностью исправен и обс...,1.8 LTR,190 N12,бензин,87000,2014,A6,...,1290000.0,2015,1097391980,SEDAN ROBOT 1.8,роботизированная,2 владельца,NaN,Оригинал,передний,Левый
9,лифтбек,BMW,синий,"Разносторонний характер, объединяющий в себе р...",2.0 LTR,184 N12,дизель,171644,2011,3ER,...,1427000.0,2014,1099563408,LIFTBACK AUTOMATIC 2.0,автоматическая,1 владелец,NaN,Оригинал,полный,Левый


In [10]:
profile = ProfileReport(df=train, title='EDA')

In [11]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/36 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# for _ in range(100):
#     print(train.description.values[random.randint(1,1000)])
#     clear_output(wait=True)
#     time.sleep(2)

# Preprocessing

In [189]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

(6682, 23) (1671, 23) (8353, 23)


In [190]:
stop_phrases = [
".*криминалистическ.*проверк.*",
".*юридическ.*проверк.*",
".*юридич.*чистот.*",
".*комплекс.*диагност.*",
".*мы.*гарантируем.*",
".*мы.*готовы.*",
".*наша.*компания.*",
".*партнер.*",
".*предпродажн.*подготовк.*",
".*победитель.*",
".*кредит.* \w+ \d+.*",
".*рады.*вас.*",
".*выгод.*процент.*",
".*автокредит.*",
".*взнос.*",
".*ждем.*вас.*",
".*специальный.*условия.*",
".*представлен.*",
".*уточн.*менедж.*",
".*уточн.*телеф.*",
".*на.*сайт.*",
".*нам.*довер.*",
".*страхов.*",
".*каско.*",
".*осаго.*",
".*трейд.*",
".*trade.*",
".*тест.*драйв.*",
".*техн.*поддержк.*",
".*холдинг.*",
".*брендов.*",
".*в.* наличии.*",
".*ассортимент.*",
".*большой.*выбор.*",
".*уникальн.*условия.*",
".*финансиров.*",
".*комиссия.*",
".*комфорт: .*",
".*круглосуточно.*",
".*индивидуальные.*условия.*",
".*первоначальный.*взнос.*",
".*автокредит.*",
".*погашен.*",
".*военн.*билет.*",
".*большинств.*регион.*",
".*консультант.*",
".*автодилер года.*",
".*ежеднев.*",
".*более.*подробн.*",
".*возмож.*банк.*",
".*безнал.*расч.*",
".*пакет.*докум.*",
".*оформлени.*",
".*цен.*соотв.*",
".*ссылк.*",
".*выездная.*оценка.*",
".*услуг.*",
".*программ.*",
".*менеджер.*",
".*узна.*подробн.*",
".*кредитован.*:",
".*комплектация*:",
".*предлагаем.*:",
".*прошли.*:",
".*международн.*:",
".*крупнейш.*дилер.*",
".*вы.*получаете.*",
".*для.*вас.*",
".*груп.*компаний.*",
".*выгода.*до.*",
".*двум*докум.*",
]

In [191]:
def get_enginePower(enginePower):
    regex = re.compile('\d+')
    return float(regex.findall(enginePower)[0])

def get_engineDisplacement(engineDisplacement):
    if engineDisplacement == "undefined LTR":
        return 2.0
    regex = re.compile('\d+.\d+')
    return float(regex.findall(engineDisplacement)[0])


def add_custom_columns(input_data, mean_encoding_columns=['brand', 'name']):
    
    data = input_data.copy()
    data['log_mileage'] = data['mileage'].apply(np.log)
    data['log_modelDate'] = data['modelDate'].apply(np.log)
    data['price'] = data['price'].apply(np.log)
    data['enginePower'] = data['enginePower'].map(get_enginePower)
    data['engineDisplacement'] = data['engineDisplacement'].map(get_engineDisplacement)
    data['vehicleConfiguration'] = data['vehicleConfiguration'].str.split(' ').apply(lambda x: x[0])
    
    curr_year = datetime.now().year
    data['powerPerDisplacement'] = data['enginePower']/data['engineDisplacement']
    data['mileagePerYear'] = data['mileage']/(curr_year - data['modelDate'].apply(lambda x: x if x!=curr_year else (curr_year-1)))
    
    for column in mean_encoding_columns:
        data[f"{column}_price_mean"] = 1
        data[f"{column}_price_mean"] = data[column].map(data.query("sample == 1").groupby(column).price.mean())
    
    return data

In [192]:
def clean_description(description_column):
    
    sentences = description_column.split('\n')
    if len(sentences) < 5:
        return description_column
    
    for i in range(len(sentences)-1, -1, -1):
        if len(sentences[i]) < 5:
            del sentences[i]
            continue
        sentence_lower = sentences[i].lower()
        for st_wrd in stop_phrases:
            match = re.search(st_wrd, sentence_lower)
            if match == None:
                continue
            elif (match[0] != "") and (sentences[i] in sentences):
                del sentences[i]
                break

    bullit_list = []
    for i in range(len(sentences)-1,-1,-1):
        if not sentences[i]:
            bullit_list.append((i, ''))
            continue

        bullit_prefix_match = re.search('[^\w]+?(?=[\w])', sentences[i])
        car_code_prefix_match = re.search('^[A-Z0-9_]+[\S]+', sentences[i])
        if bullit_prefix_match:
            if bullit_prefix_match[0] != "": 
                bullit = bullit_prefix_match[0]
        elif car_code_prefix_match:
            if car_code_prefix_match[0] != "":
                bullit = 'AAAAAAAAAAA'   
        else:
            bullit = ""
        bullit_list.append((i, bullit))
    bullit_list.reverse()

    comparison_list = []
    repeats_list = []
    bullit_repeats = 0
    for index, sent_tuple in enumerate(bullit_list):
        comparison_list.append(sent_tuple[1])
        if index < 1:
            repeats_list.append((0,0))
            continue
        if comparison_list[-1] == comparison_list[-2] and comparison_list[-2] != '':
            bullit_repeats += 1
        else:
            bullit_repeats = 0
        repeats_list.append((index, bullit_repeats))

    delete_index_list = []

    repeats_values = [repeats[1] for repeats in repeats_list]
    while True:
        if not repeats_values:
            break
        max_repeats = max(repeats_values)
        if max_repeats == 0:
            break
        max_repeats_index = repeats_values.index(max_repeats)
        if (max_repeats+1) >= 11:
            delete_indexes = [ind for ind in range(max_repeats_index-max_repeats, max_repeats_index+1)]
            delete_index_list += delete_indexes
            for del_ind in sorted(delete_indexes, reverse=True):
                repeats_values[del_ind] = 0
        else:
            break

    for del_ind in sorted(delete_index_list, reverse=True):
        del sentences[del_ind]

        
    return "\n".join(sentences)

In [193]:
data = add_custom_columns(data)

data.drop('Владение', axis=1, inplace=True)
data.drop(['vehicleConfiguration'], axis=1, inplace=True)

numerical_features = list(data.select_dtypes(include=['int64', 'float64']).columns)
numerical_features.remove('price')
categorical_features = list(data.select_dtypes(include=['object']).columns)
categorical_features.remove('description')
text_features = 'description'

scaler = MinMaxScaler()
for num_column in numerical_features:
    data[num_column] = scaler.fit_transform(data[[num_column]])[:,0]
    
for cat_column in categorical_features:
    data[cat_column] = data[cat_column].astype('category').cat.codes
data = pd.get_dummies(data, columns=categorical_features, dummy_na=False)

data['description'] = data['description'].apply(clean_description)
megatext = "\n".join(data.description.values.tolist())
stop_words = list(set(re.findall('[^\w]', megatext)))
data['description'] = data['description'].apply(lambda x: [i for i in x.lower().split() if i not in stop_words]).str.join(' ')

In [194]:
data_ml = data.drop('description', axis=1)

In [373]:
X, y = data_ml.query('sample == 1').drop('price', axis=1).drop('sample', axis=1), data_ml.query('sample == 1').price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=42)

In [374]:
model = CatBoostRegressor(iterations = 6000,
                          depth=4,
                          learning_rate = 0.05,
                          random_seed = 42,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=1000,
         use_best_model=True,
         plot=True,
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0407246	test: 0.0414486	best: 0.0414486 (0)	total: 4.41ms	remaining: 26.4s
1000:	learn: 0.0074493	test: 0.0081071	best: 0.0081070 (999)	total: 3.87s	remaining: 19.3s
2000:	learn: 0.0064562	test: 0.0077265	best: 0.0077255 (1987)	total: 8.15s	remaining: 16.3s
3000:	learn: 0.0058585	test: 0.0076206	best: 0.0076198 (2994)	total: 12.7s	remaining: 12.7s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.007587565247
bestIteration = 3432

Shrink model to first 3433 iterations.


In [375]:
test_predict_catboost = model.predict(X_test)
mape_log = (abs(np.exp(test_predict_catboost) - np.exp(y_test))/np.exp(y_test)).mean()

In [376]:
print(f"TEST mape: {mape_log*100:0.2f}%")

TEST mape: 10.84%


In [244]:
# dictionary = Dictionary(occurence_lower_bound=0)
# tokenized = Tokenizer()
# dictionary.fit(data.description.values, tokenizer=tokenized)

In [247]:
# len(dictionary.apply(data.description.apply(word_tokenize))[0])

28